In [60]:
from math import sqrt
import numpy as np
from sympy import Float, init_printing, Array, relational, symbols, Matrix
init_printing()

#### Input Data

In [61]:
def target_function(x1,x2):
    return 2*(x1 - 1)**2 + x1*x2 + 6*x2**2


def df_dx1(x1, x2):
    return 4*x1 + x2 - 4


def df_dx2(x1, x2):
    return x1 + 12*x2


ANALYTICAL_GRADIENT = [df_dx1, df_dx2]
START_POINT = [5.2, 5.2]
ANALYTICAL_HESSE = [[lambda x, y: 4, lambda x, y: 1], [lambda x, y: 1, lambda x, y: 12]]

x_min, f_min = symbols('x_min f_min')

#### Helpers

In [62]:
def calculate_gradient(analytical_gradient, point):
    """
    :param analytical_gradient: list of lambdas (analytical gradient)
    :param point:               list of numbers (current point)
    :return:                    list of numbers (calculated gradient)
    """
    values_gradient = []
    for i in range(len(analytical_gradient)):
        values_gradient.append(analytical_gradient[i](*point))
    return values_gradient


def calculate_anti_gradient(gradient):
    """
    :param gradient: list of numbers (gradient values)
    :return:         list of numbers (anti-gradient values)
    """
    anti_gradient = []
    for i in range(len(gradient)):
        anti_gradient.append(-1 * gradient[i])
    return anti_gradient


def calculate_gradient_norm(gradient):
    res = 0
    for i in range(len(gradient)):
        res += gradient[i]**2
    return sqrt(res)


def calculate_optimal_step(s_vector, gradient_values, hesse):
    """
    :param s_vector:        list of numbers       (the direction of the greatest decline of the function)
    :param gradient_values: list of numbers       (gradient in point)
    :param hesse:           multidimensional list (hesse matrix)
    :return:                number                (optimal step)
    """
    s = np.array(s_vector)
    g = np.array(gradient_values)
    h = np.array(hesse)

    a = g.transpose().dot(s)
    b = s.transpose().dot(h).dot(s)

    return -1 * a / b


def calculate_hesse(analytical_hesse, point):
    result = []
    for i in range(len(analytical_hesse)):
        result.append([])
        for j in range(len(analytical_hesse[i])):
            result[i].append(analytical_hesse[i][j](*point))
    return result

#### Fletcher-Rives's method

In [63]:
def calculate_new_direction_fr(anti_gradient, prev_gradient, current_gradient, prev_direction):
    """
    Calculating of new direction by Fletcher-Rives' method
    :param anti_gradient:       list of numbers (anti-gradient values)
    :param prev_gradient:       list of numbers (gradient values in prev point)
    :param current_gradient:    list of numbers (gradient values in current point)
    :param prev_direction:      list of numbers (direction vector in prev point)
    :return: 
    """
    ag = np.array(anti_gradient, dtype=float)
    pg = np.array(prev_gradient, dtype=float)
    cg = np.array(current_gradient, dtype=float)
    pd = np.array(prev_direction, dtype=float)

    a = cg.transpose().dot(cg)
    b = pg.transpose().dot(pg)
    if b == 0:
        return []
    return np.add(ag, (a / b) * pd)


def fletcher_rives_method(analytical_gradient, start_point, hesse, epsilon):
    """
    Minimization of function by Fletcher-Rives' method
    :param analytical_gradient: list of lambdas     (analytical gradient)
    :param start_point:         list of numbers     (start point)
    :param hesse:               2d list of lambdas  (analytical hesse' matrix)
    :param epsilon:             float               (accuracy)
    :return:                    list of points and list of gradients norms for each iteration
    """
    points = []
    gradient_norms = []
    point = start_point.copy()
    gradient = calculate_gradient(analytical_gradient, point)
    direction = calculate_anti_gradient(gradient)
    step = calculate_optimal_step(direction, gradient, calculate_hesse(hesse, point))
    gradient_norm = np.linalg.norm(np.array(gradient))

    points.append(point)
    gradient_norms.append(gradient_norm)

    while gradient_norm > epsilon:
        point =  np.add(np.array(point), (step * np.array(direction)))
        new_gradient = calculate_gradient(analytical_gradient, point)
        new_anti_gradient = calculate_anti_gradient(new_gradient)

        direction = calculate_new_direction_fr(new_anti_gradient, gradient, new_gradient, direction)
        if len(direction) == 0:
            print("To small value of gradient norm")
            break
        gradient = new_gradient
        gradient_norm = np.linalg.norm(np.array(new_gradient))
        step = calculate_optimal_step(direction, gradient, calculate_hesse(hesse, point))
        gradient_norms.append(gradient_norm)
        points.append(list(point))

    return points, gradient_norms


fr_points_array, fr_gradient_norms_array = fletcher_rives_method(
    ANALYTICAL_GRADIENT,
    START_POINT,
    ANALYTICAL_HESSE,
    0.01
)

##### Fletcher-rives method result

In [64]:
relational.Eq(x_min, Array(fr_points_array[len(fr_points_array) - 1]))

xₘᵢₙ = [1.02127659574468  -0.0851063829787198]

In [65]:
relational.Eq(f_min, Float(str(target_function(*fr_points_array[len(fr_points_array) - 1])), 5))


#### DFP method


In [66]:
def calculate_new_matrix_a(prev_matrix, delta_x, delta_g):
    a = Matrix(prev_matrix)
    x = Matrix(delta_x)
    g = Matrix(delta_g)

    return np.array(
        a +
        ((x * x.transpose()) / (x.transpose() * g)[0]) -
        (((a * g) * g.transpose()) * a) / (((g.transpose() * a) * g)[0]))


def dfp_method(analytical_gradient, start_point, hesse, epsilon):
    """
    Minimization of function by DFP method
    :param analytical_gradient: list of lambdas     (analytical gradient)
    :param start_point:         list of numbers     (start point)
    :param hesse:               2d list of numbers  (hesse's matrix)
    :param epsilon:             float               (accuracy)
    :return: list of points and list of gradients norms for each iteration
    """
    points = []
    gradient_norms = []
    point = np.array(start_point.copy())
    gradient = np.array(calculate_gradient(analytical_gradient, point))
    a_matrix = np.array(np.eye(len(analytical_gradient)))

    direction = -1 * a_matrix.dot(gradient)
    step = calculate_optimal_step(direction, gradient, hesse)
    gradient_norm = calculate_gradient_norm(gradient)

    points.append(point)
    gradient_norms.append(gradient_norm)

    while gradient_norm > epsilon:
        new_point =  np.add(np.array(point), (step * np.array(direction)))
        new_gradient = np.array(calculate_gradient(analytical_gradient, new_point))

        delta_g = new_gradient - gradient
        delta_x = new_point - point

        a_matrix = np.array(calculate_new_matrix_a(a_matrix, delta_x, delta_g))
        direction = -1 * a_matrix.dot(new_gradient)

        gradient = new_gradient
        gradient_norm = calculate_gradient_norm(new_gradient)
        step = calculate_optimal_step(direction, gradient, hesse)
        point = new_point

        gradient_norms.append(gradient_norm)
        points.append(list(point))

    return points, gradient_norms

dfp_points_array, dfp_norms_array = dfp_method(
    ANALYTICAL_GRADIENT,
    START_POINT,
    calculate_hesse(ANALYTICAL_HESSE, [0, 0]),
    0.01
)

##### Fletcher-rives method result

In [67]:
relational.Eq(x_min, Array(dfp_points_array[len(dfp_points_array) - 1]))

xₘᵢₙ = [1.02127659574468  -0.0851063829787248]

In [68]:
relational.Eq(f_min, Float(str(target_function(*dfp_points_array[len(dfp_points_array) - 1])), 5))